Участники, посещающие форум-встречи чаще продлевают участие на следующий период
-------------------------



## setups

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.options.plotting.backend = "plotly"   # 'matplotlib'  'plotly'
print(pd.get_option("plotting.backend"))

from pandas import Timestamp

import plotly.express as px

import numpy as np
from scipy.stats import chi2_contingency

plotly


In [57]:
pd.options.plotting.backend = "plotly"
# pd.options.plotting.backend = "matplotlib"
pd.get_option("plotting.backend")   # 'matplotlib'  'plotly'

'plotly'

Полезное

https://plotly.com/python/line-charts/  
https://plotly.com/python/histograms/  
https://plotly.com/python/bar-charts/  
https://plotly.com/python/2D-Histogram/     (тепловые)
https://plotly.com/python/imshow/
https://plotly.com/python-api-reference/generated/plotly.express.imshow.html

px.colors.sequential.swatches_continuous()  

In [26]:
# функция для принятие решения об отклонении нулевой гипотезы

def decision_hypothesis(p, alpha = 0.05):
  if p <= alpha:
      print(f'P-значение {p:.3f} меньше, чем заданный уровень значимости {alpha}. \n --> 1️⃣ Отвергаем нулевую гипотезу в пользу альтернативной.'.format(alpha))
  else:
      print(f'P-значение {p:.3f} больше, чем заданный уровень значимости {alpha}. \n --> 0️⃣ У нас нет оснований отвергнуть нулевую гипотезу.'.format(alpha))


# Загрузка датасетов

### Группировка по периодам

In [2]:

# Начато в базе закончено в Python
# Загружаем и применяем ограничение датой выгрузки

members_plus_fmeets_by_mper_id = pd.read_csv('data\members_inner_fmeets_by_mper_id.csv', parse_dates=['start_membership_date', 'end_membership_date'])
    
# members_plus_fmeets_by_mper_id = pd.read_csv('data\members_plus_fmeets_by_mper_id.csv', parse_dates=['start_membership_date', 'end_membership_date'])
# members_and_fmeets_by_mper = pd.read_csv('data\members_and_fmeets_by_mper_conc.csv', parse_dates=['start_membership_date', 'end_membership_date'])
# members_and_fmeets_by_mper = pd.read_csv('data\members_and_fmeets_by_mper.csv', parse_dates=['start_membership_date', 'end_membership_date'])
# members_and_fmeets_by_mper.sample(3)


# Применяем ограничение по дате выгрузки
members_plus_fmeets_by_mper_id = members_plus_fmeets_by_mper_id[members_plus_fmeets_by_mper_id.end_membership_date < Timestamp('2024-03-04 00:00:00')]

members_plus_fmeets_by_mper_id.tail(1)

,mper_id,stage,start_membership_date,end_membership_date,user_id,contact_id,churn,membership_days,membership_months,membership_cut,seq_number,meeting_id,at_meeting_sum,at_meeting_ewm,at_meeting_stab,encouraged,mper_num,is_continued
2749,105662,Завершено выходом из клуба,2024-01-04,2024-02-21,2554.0,2736,True,48,1,True,0.0,1,0.0,0.0,0.0,0,1,False


In [3]:
# # # Границы исследования (по дате выгрузки)
# # # Timestamp('2018-02-11 00:00:00'), Timestamp('2024-03-03 00:00:00') 

# Временные границы датасета
members_plus_fmeets_by_mper_id.start_membership_date.min(),\
members_plus_fmeets_by_mper_id.end_membership_date.max()

# (Timestamp('2018-02-11 00:00:00'), Timestamp('2024-10-02 00:00:00'))

(Timestamp('2018-02-11 00:00:00'), Timestamp('2024-03-03 00:00:00'))

In [43]:
# # Применяем ограничение по дате выгрузки
# members_plus_fmeets_by_mper_id = members_plus_fmeets_by_mper_id[members_plus_fmeets_by_mper_id.end_membership_date < Timestamp('2024-03-04 00:00:00')]
# members_plus_fmeets_by_mper_id.sort_values('end_membership_date').tail(1)
# # members_plus_fmeets_by_mper_id[members_plus_fmeets_by_mper_id.end_membership_date < Timestamp('2024-03-03 00:00:00')]

,mper_id,stage,start_membership_date,end_membership_date,user_id,member_id,contact_id,churn,membership_days,membership_months,membership_cut,duration,seq_number,forum_id,forum_year,city_id,notification_one_week_before_was_send,notification_one_day_before_was_send,participants_number,curator_id,bitrix_id,mper_meetings_count,at_meeting_sum,will_come_sum,refuse_sum,is_continued,encouraged,mper_num
1577,89780,Форум подобран (активное участие),2023-03-04,2024-03-03,1318.0,1816.0,1212,False,365,12,False,4.0,8.0,27.0,0.0,1.0,7,8,7.0,883.0,0.0,9,4.0,2.0,3.0,True,0,3


In [4]:
members_plus_fmeets_by_mper_id.stage.value_counts()

stage
Год членства завершен                1815
Завершено выходом из клуба            453
Форум подобран (активное участие)      11
Участие без форумов                     5
Замена                                  2
Ожидание ОФ                             1
Name: count, dtype: int64

### Группировка по контактам (после группировки по периодам)

In [2]:
# members_inner_fmeets_by_mper_contact
# Ограничение по дате выгрузки уже применено 

members_plus_fmeets_by_mper_contact = pd.read_csv('data\members_inner_fmeets_by_mper_contact.csv', parse_dates=['start_membership_date', 'end_membership_date'])
members_plus_fmeets_by_mper_contact.tail(1)

,start_membership_date,end_membership_date,user_id,contact_id,encouraged,mper_id,churn,is_continued,membership_days,membership_months,membership_cut,seq_number,meeting_id,at_meeting_sum,will_come_sum,refuse_sum
1122,2024-01-04,2024-02-21,2554.0,2736,0,1,True,0,48,1,True,0.0,1,0.0,0.0,0.0


#### Статистика

In [87]:
# Временные границы исследования
members_and_fmeets_by_mper['start_membership_date'].min(), members_and_fmeets_by_mper['end_membership_date'].max()

(Timestamp('2019-12-31 00:00:00'), Timestamp('2024-03-03 00:00:00'))

In [111]:
# Периодов concated
len(members_and_fmeets_by_mper)
# 417

1104

In [112]:
# Уникальных пользователей
# members_and_fmeets_by_mper.user_id.nunique()
members_and_fmeets_by_mper.contact_id.nunique()
# 399

970

In [114]:
# Ушедших
members_and_fmeets_by_mper[members_and_fmeets_by_mper['churn'] == True].contact_id.nunique()
# members_and_fmeets_by_mper[members_and_fmeets_by_mper['is_continued'] == False].contact_id.nunique()
# 17


133

In [113]:
# Посещавших форум-встречи
members_and_fmeets_by_mper[members_and_fmeets_by_mper['at_meeting_sum'] > 0].contact_id.nunique()
# 136

332

# Общий анализ датасетов

In [5]:
# Группировка по ПЕРИОДАМ, Первая и последняя строчки
display(members_plus_fmeets_by_mper_id.head(1), members_plus_fmeets_by_mper_id.tail(1))

,mper_id,stage,start_membership_date,end_membership_date,user_id,contact_id,churn,membership_days,membership_months,membership_cut,seq_number,meeting_id,at_meeting_sum,at_meeting_ewm,at_meeting_stab,encouraged,mper_num,is_continued
0,5674,Год членства завершен,2018-02-11,2019-01-27,1261.0,822,False,350,11,False,2.0,7,7.0,1.0,0.0,0,1,True


,mper_id,stage,start_membership_date,end_membership_date,user_id,contact_id,churn,membership_days,membership_months,membership_cut,seq_number,meeting_id,at_meeting_sum,at_meeting_ewm,at_meeting_stab,encouraged,mper_num,is_continued
2749,105662,Завершено выходом из клуба,2024-01-04,2024-02-21,2554.0,2736,True,48,1,True,0.0,1,0.0,0.0,0.0,0,1,False


In [15]:
# Группировка по УЧВСТНКАМ, Первая и последняя строчки
display(members_plus_fmeets_by_mper_contact.head(1), members_plus_fmeets_by_mper_contact.tail(1))
# members_plus_fmeets_by_mper_id.iat[1, 2,]


,start_membership_date,end_membership_date,user_id,contact_id,encouraged,mper_id,churn,is_continued,membership_days,membership_months,membership_cut,seq_number,meeting_id,at_meeting_sum,will_come_sum,refuse_sum
0,2018-02-11,2023-12-15,1261.0,822,10,5,False,5,2129,69,False,3.0,56,46.0,1.0,9.0


,start_membership_date,end_membership_date,user_id,contact_id,encouraged,mper_id,churn,is_continued,membership_days,membership_months,membership_cut,seq_number,meeting_id,at_meeting_sum,will_come_sum,refuse_sum
1122,2024-01-04,2024-02-21,2554.0,2736,0,1,True,0,48,1,True,0.0,1,0.0,0.0,0.0


Датасеты собраны из таблиц:
* сделки участников клуба и соответствующие периоды
* несколько таблиц для нахождения соответствий различных ID учасников
* таблица о встречах клуба
* таблица о типах встреч
* таблица с посещениями встреч участниками клуба

Создано неколько дополнительных признаков.

Дополнительные признаки по периодам членства:
* количества дней в периоде
* количества месяцев в периоде
* количества дополнительных месяцев (>12)
* досрочного прекращения периода (<10)
* номер последней встречи в периоде
* признак оттока
* было ли продлено членство после периода
* номер периода у участника (1-6)

Дополнительные признаки по встречам (отобраны только состоявшиеся):
* количество встреч в периоде
* количество посещенных встреч
* количество встреч с отметкой "приду"
* количество встреч с отметкой "не пришел"



### Общая статистика

In [17]:
# Временные границы после группировок
members_plus_fmeets_by_mper_contact.start_membership_date.min(),\
members_plus_fmeets_by_mper_contact.end_membership_date.max()

(Timestamp('2018-02-11 00:00:00'), Timestamp('2024-03-03 00:00:00'))

Это - первое начало периода и последнее завершение периода.  
Последняя дата -- перед выгрузкой данных.

Многие отсеченные периоды начались еще в 23-м году и можно предполагать, что большинство их них будет продлено, но факта продления еще нет, и мы их не можем рассматривать. 

| Параметр | Значение |
| --- | --- |
| Периодов после соединения и группировки | 2658 |
| Уникальных участников | 1226 |
| Отток | 514 |
| Участников с отметкой "на встрече" в периодах | 799 |
| Участников с отметкой "приду" в периодах   | 262 |
| Участников с отметкой "не пришел" в периодах   |691  |
| Среднее количество посещений за период  | 4,2 |

`TOP количеств встреч, посещенных за период`

| Встреч посетил | Как часто встречается |
| --- | --- |
| 1 | 881 |
| 9 | 321 |
| 10  |  316 |
| 11  |  230 |
| 7   |  178 |
| 8   |  169 |
| 12  |  135 |
| 6   |  123 |
| 5   |   88 |
| 4   |   85 |
| 3   |   73 |
| 2   |   59 |

В большом отрыве -- единственное посещение.  
9,10,11 -- самые устойчивые количества.

In [29]:
# Месяцы начала членства и их количество
members_plus_fmeets_by_mper_contact.start_membership_date.dt.month.hist(title='Месяцы начала членства и их количество')

Нименьший приток в целом наблюдается в мае и августе.  
Наибольший приток в целом -- к концу года.  

### Количество месяцев в клубе и количество посещенных встреч

In [8]:
# Количество МЕСЯЦЕВ в клубе, количество ПОСЕЩЕННЫХ встреч и ПРОДЛЕНИЯ
px.scatter(members_plus_fmeets_by_mper_contact, x='membership_months', y='at_meeting_sum', color='is_continued', color_continuous_scale='turbo', title='Количество МЕСЯЦЕВ в клубе, количество ПОСЕЩЕННЫХ встреч и ПРОДЛЕНИЯ') # , violinmode='overlay'  suspectedoutliers


In [19]:
# Количество МЕСЯЦЕВ в клубе, количество ПОСЕЩЕННЫХ встреч и НОМЕР ПЕРИОДА
px.scatter(members_plus_fmeets_by_mper_contact, x='membership_months', y='at_meeting_sum', color='mper_id', color_continuous_scale='turbo', title='Количество МЕСЯЦЕВ в клубе, количество ПОСЕЩЕННЫХ встреч и № ПЕРИОДА') # , violinmode='overlay'  suspectedoutliers


In [8]:
# Количество ДНЕЙ в клубе, количество ПОСЕЩЕННЫХ встреч и факт ОТТОКА
px.scatter(members_plus_fmeets_by_mper_contact, x='membership_days', y='at_meeting_sum', color='churn', color_discrete_sequence=[px.colors.qualitative.Plotly[2], px.colors.qualitative.Plotly[1]], title='Количество ДНЕЙ в клубе, количество ПОСЕЩЕННЫХ встреч и факт ОТТОКА') # , violinmode='overlay'  suspectedoutliers


#### Проверка

In [4]:
# members_plus_fmeets_by_mper_id    members_plus_fmeets_by_mper_contact
members_plus_fmeets_by_mper_id.columns, members_plus_fmeets_by_mper_contact.columns

(Index(['mper_id', 'stage', 'start_membership_date', 'end_membership_date',
        'user_id', 'contact_id', 'churn', 'membership_days',
        'membership_months', 'membership_cut', 'seq_number', 'meeting_id',
        'at_meeting_sum', 'will_come_sum', 'refuse_sum', 'encouraged',
        'mper_num', 'is_continued'],
       dtype='object'),
 Index(['start_membership_date', 'end_membership_date', 'user_id', 'contact_id',
        'encouraged', 'mper_id', 'churn', 'is_continued', 'membership_days',
        'membership_months', 'membership_cut', 'seq_number', 'meeting_id',
        'at_meeting_sum', 'will_come_sum', 'refuse_sum'],
       dtype='object'))

In [29]:
members_plus_fmeets_by_mper_contact.query('is_continued == 1 and mper_id == 1').sort_values('contact_id')[:20]
# members_plus_fmeets_by_mper_contact.query('is_continued == 2 and mper_id == 2')#[:20]
# members_plus_fmeets_by_mper_contact.query('is_continued == 3 and mper_id == 3')#[:20]


,start_membership_date,end_membership_date,user_id,contact_id,encouraged,mper_id,churn,is_continued,membership_days,membership_months,membership_cut,seq_number,meeting_id,at_meeting_sum,will_come_sum,refuse_sum
1083,2023-01-02,2024-01-01,2355.0,332,0,1,False,1,364,12,False,3.0,2,0.0,2.0,0.0
612,2021-08-02,2022-08-02,1001.0,378,0,1,False,1,365,12,False,0.0,1,0.0,0.0,0.0
911,2022-03-29,2023-03-28,1719.0,668,0,1,False,1,364,12,False,1.0,6,1.0,0.0,5.0
946,2022-05-28,2023-05-27,1717.0,670,0,1,False,1,364,12,False,1.0,12,9.0,0.0,3.0
956,2022-06-05,2023-06-04,1737.0,672,0,1,False,1,364,12,False,1.0,5,3.0,0.0,2.0
866,2022-02-01,2023-01-31,1697.0,734,0,1,False,1,364,12,False,0.0,1,0.0,0.0,0.0
909,2022-03-28,2023-03-27,1718.0,796,0,1,False,1,364,12,False,10.0,7,5.0,1.0,1.0
481,2021-01-25,2022-01-25,1191.0,1196,0,1,False,1,365,12,False,0.0,1,0.0,0.0,0.0
644,2021-09-09,2022-09-09,1409.0,1252,0,1,False,1,365,12,False,0.0,11,11.0,0.0,0.0
924,2022-04-25,2023-04-24,1763.0,1380,0,1,False,1,364,12,False,0.0,8,6.0,1.0,1.0


In [26]:
# members_plus_fmeets_by_mper_id.query('contact_id == 378 or contact_id == 1196 or contact_id == 1252 or contact_id == 1738 or contact_id == 1848').sort_values('contact_id')
members_plus_fmeets_by_mper_id.query('contact_id == 2300 or contact_id == 1392 or contact_id == 1432 or contact_id == 1568 or contact_id == 1536').sort_values('contact_id')


,mper_id,stage,start_membership_date,end_membership_date,user_id,contact_id,churn,membership_days,membership_months,membership_cut,seq_number,meeting_id,at_meeting_sum,will_come_sum,refuse_sum,encouraged,mper_num,is_continued
1447,2000,Год членства завершен,2022-03-21,2023-03-20,1720.0,1392,False,364,12,False,0.0,1,0.0,0.0,0.0,0,1,True
1448,2476,Год членства завершен,2022-03-22,2023-03-21,1708.0,1432,False,364,12,False,0.0,1,0.0,0.0,0.0,0,1,True
1452,1844,Год членства завершен,2022-03-25,2023-03-24,1721.0,1536,False,364,12,False,4.0,4,4.0,0.0,0.0,0,1,True
1449,1876,Год членства завершен,2022-03-23,2023-03-22,1723.0,1568,False,364,12,False,6.0,10,10.0,0.0,0.0,0,1,True
1446,3946,Год членства завершен,2022-03-21,2023-03-20,1709.0,2300,False,364,12,False,0.0,1,0.0,0.0,0.0,0,1,True


In [22]:
# Был на встречах
members_plus_fmeets_by_mper_id[members_plus_fmeets_by_mper_id["contact_id"] == 1252]


,mper_id,stage,start_membership_date,end_membership_date,user_id,contact_id,churn,membership_days,membership_months,membership_cut,seq_number,meeting_id,at_meeting_sum,will_come_sum,refuse_sum,encouraged,mper_num,is_continued
1000,1454,Год членства завершен,2021-09-09,2022-09-09,1409.0,1252,False,365,12,False,0.0,11,11.0,0.0,0.0,0,1,True


#### вывод

Приближаясь к нашему лесу встреч, посмотрим на группы деревьев.  
Увидим зиму, весну, лето и осень.

Взят датасет, сгуппированный по участникам.  
На этом графике можно увидеть общую картину посещаемости форум-встреч на протяжении разных периодов членства в клубе.

Каждая точка -- это участник клуба.

Присутствуют 3 признака участников:
- стаж в месяцах (ось 'x')
- количество посещений у участников с таким стажем (ось 'y')
- количество продлений членства (цвет)

Наблюдения:
* У тех, кто был в клубе менее 10 месяцев почти нет посещений. Они и не продлевали.
* Цветные столбики возле 12, 24, 36, 48, 60 месяцев говорят о том, что их период заканчивался в регулярный срок.
* Остальные рассеянные точки -- видимо, поощренные участники или досрочно прекратившие период.
* Самый разнообразный год по количеству посещений -- 4-й (желтое продление): посетившие 0-10 раз в нем еще заметны.
* В красном продлении таковых уже почти нет.

Можно ли сделать предварительный вывод о наличии связи посещаемости форум-встреч с продлением?  
Думаю, да: с увеличением стажа посещаемость уверенно растет и даже отрывается от земли!

# Периоды пользователей, посещения формум-встреч, временная оценка посещений и факты продлений периода

Обратимся к датасету сгруппированному по периодам.  

Заглянув в периоды, мы сможем увидеть много интересных деталей.  
Ведь решения о продлении членства зреют и принимаются в конкретных периодах.  
Здесь нас интересует, будет продлен период или нет.  

## Факты посещения форум встреч

### Участник | факт посещения в году --> продление
ПРОВЕРИМ  
`Есть ли связь между фактом посещения мероприятий в каждый период и фактом продлением участия после него?`

In [20]:
# Построим таблицу сопряженности (для теста)
cr_tab = pd.crosstab(members_plus_fmeets_by_mper_id.at_meeting_sum.astype(bool), members_plus_fmeets_by_mper_id.is_continued)
# cr_tab = pd.crosstab(members_plus_fmeets_by_mper_id.at_meeting_sum.astype(bool), members_plus_fmeets_by_mper_id.is_continued, normalize=True, margins=True).round(2)
cr_tab

is_continued,False,True
at_meeting_sum,,
False,300,411
True,153,1423


In [21]:
# Построим таблицу сопряженности (для визуализации)
cr_tab_ = cr_tab.rename(index={False: 'нет', True: 'да', 'All': 'итог'}, columns={False: 'нет', True: 'да', 'All': 'итог'}) #.style.background_gradient()
# (index=pd.Index(['нет', 'да']))
# cr_tab.style.background_gradient()
# cr_tab.style.background_gradient(cmap='YlOrBr', vmin=0, vmax=1)

# index=pd.Index(['Tumour (Positive)', 'Non-Tumour (Negative)'], name='Actual Label:'),
#                   columns=pd.MultiIndex.from_product([['Decision Tree', 'Regression', 'Random'],['Tumour', 'Non-Tumour']], names=['Model:', 'Predicted:']))
# df.style

In [22]:
# Построим таблицу сопряженности

fig = px.imshow(cr_tab_, text_auto=True, color_continuous_scale='Greens', width=400, height=300) # , color_continuous_midpoint=0    'turbo'    , aspect="auto"
fig.update_xaxes(side="top")
fig.update_xaxes(title_text="Период продлен")
fig.update_yaxes(title_text="Был на встречах")
fig.show()
# px.density_heatmap(cr_tab, text_auto=True, color_continuous_scale='Greens', width=400, height=300) # , color_continuous_midpoint=0    'turbo'

In [24]:
# Что означают цыфры в таблице
print(f'В текущем периоде:\n')
print(f'Посещали    🤝✅  и продлили     🔃✅ членство далее:  {cr_tab[True][1]}')
print(f'Не посещали 🤝❌, но продлили    🔃✅ членство далее:  {cr_tab[True][0]}')
print(f'Не посещали 🤝❌  и не продлили  🔃❌ членство далее:  {cr_tab[False][0]}')
print(f'Посещали    🤝✅, но не продлили 🔃❌ членство далее:  {cr_tab[False][1]}')

В текущем периоде:

Посещали    🤝✅  и продлили     🔃✅ членство далее:  1423
Не посещали 🤝❌, но продлили    🔃✅ членство далее:  411
Не посещали 🤝❌  и не продлили  🔃❌ членство далее:  300
Посещали    🤝✅, но не продлили 🔃❌ членство далее:  153


In [27]:
# Применяем тест хи-квадрат

chi2, p, deg_fr, exp_freq = chi2_contingency(cr_tab)

print("Статистика хи-квадрат:", chi2.round(2))
print("P-значение:", p.round(2))
print("Ожидаемые частоты при отсутвии зависимости: \n", exp_freq.round(2))
print("-"*30)

decision_hypothesis(p)

Статистика хи-квадрат: 323.48
P-значение: 0.0
Ожидаемые частоты при отсутвии зависимости: 
 [[ 140.83  570.17]
 [ 312.17 1263.83]]
------------------------------
P-значение 0.000 меньше, чем заданный уровень значимости 0.05. 
 --> 1️⃣ Отвергаем нулевую гипотезу в пользу альтернативной.


Ожидаемые частоты (expected frequencies) - это значения, которые ожидаются в каждой ячейке таблицы сопряженности, если нулевая гипотеза **о независимости переменных** верна. 

Получили очень уверенный результат теста: даже при рассмотрении базовой черно-белой под-гипотезы о полном не-посещении,  
`связь с продлением уверенная`.

## Связь между количеством посещений форум-встреч и продлением членства после текущего периода


In [221]:
# Количество посещенных встреч теми, кто продлят членство (True) и теми, кто не продлит (False)
px.violin(members_plus_fmeets_by_mper_id, x='is_continued', y='at_meeting_sum', box=True, points='all', color_discrete_sequence=["green"], title='Количество посещений форум-встреч в период и продление членства после него') # suspectedoutliers


В целом по датасету, у тех, кто продлит членство после периода:
* срединное значение посещений -- 6
* верхняя граница 3/4 значений -- 9
* нижняя граница 3/4 значений -- 1
* концентрация наибольших вероятностей -- возле 9 и 0

У тех, кто продлит членство после периода:
* срединное значение посещений -- 0
* верхняя граница 3/4 значений -- 1
* концентрация наибольших вероятностей -- возле 0

In [18]:
# Количество выразивших намерение посещения встреч теми, кто продлят членство (True) и теми, кто не продлит (False)
px.violin(members_plus_fmeets_by_mper_id, x='is_continued', y='will_come_sum', box=True, points='all', title='Количество намерений посетить форум-встречи и продление членства после текущего периода')


## Связь между количеством посещений встреч, годом членства и продлением после него

In [ ]:
# Год членства и количество посещенных встреч теми, кто продлят членство (True) и теми, кто не продлит (False) в этот период
# at_meeting_sum  will_come_sum refuse_sum  membership_cut encouraged
# px.violin(members_plus_fmeets_by_mper_id, x='mper_num', y='at_meeting_sum', color='is_continued', box=True, points='suspectedoutliers', title='Количество посещений форум-встреч в период, год членства и продление после него') # , violinmode='overlay'  suspectedoutliers
members_plus_fmeets_by_mper_id.boxplot(x='mper_num', y='refuse_sum', color='is_continued', points='suspectedoutliers', title='Год членства и ... продлившими членство и не продлившими') # , violinmode='overlay'  suspectedoutliers


In [229]:
# Год членства и количество посещенных встреч теми, кто продлят членство (True) и теми, кто не продлит (False) в этот период
# px.violin(members_plus_fmeets_by_mper_id, x='mper_num', y='at_meeting_sum', color='is_continued', box=True, points='suspectedoutliers', title='Количество посещений форум-встреч в период, год членства и продление после него') # , violinmode='overlay'  suspectedoutliers
members_plus_fmeets_by_mper_id.boxplot(x='mper_num', y='at_meeting_sum', color='is_continued', points='suspectedoutliers', color_discrete_sequence=[px.colors.qualitative.Plotly[2], px.colors.qualitative.Plotly[1]], title='Год членства и количество посещенных встреч продлившими членство и не продлившими') # , violinmode='overlay'  suspectedoutliers


Посмотрим, как распределена посещаемость по годам членства у тех, кто продлят (True) и тех, кто не продлят (False) в конкретный период.  
Заметим, что красные коробочки пускают пузыриков с каждым годом всё меньше...

Заглянем внутрь периодов (лет) каждой группы и посмотрим на серединные значения и вероятности количеств посещений

In [230]:
# Год членства и количество посещенных встреч теми, кто продлят членство в этот период
px.violin(members_plus_fmeets_by_mper_id.query('is_continued == True'), x='mper_num', y='at_meeting_sum', box=True, points='all', color_discrete_sequence=[px.colors.qualitative.Plotly[2]], title='Год членства и количество посещенных встреч теми, кто ПРОДЛИТ членство в этот период') # , violinmode='overlay'  suspectedoutliers


В первый год вероятности посещения все еще тяготеют к 0. 
Но вероятность возле 9 посещений тоже крепка.

Далее вероятности всё более перетекают к 8-9.

In [226]:
# Год членства и количество посещенных встреч теми, кто не продлит членство в этот период
px.violin(members_plus_fmeets_by_mper_id.query('is_continued == False'), x='mper_num', y='at_meeting_sum', box=True, points='all', color_discrete_sequence=[px.colors.qualitative.Plotly[1]], title='Год членства и количество посещенных встреч теми, кто НЕ ПРОДЛИТ членство в этот период') # , violinmode='overlay'  suspectedoutliers


Первый год у не продливших его все-же довольно насыщен посещениями.  
У не продливших второй год уже крепнет тенденция к не-посещению.  
Далее и сами не-продлевающие постепенно иссякают...

Можно ли понять по посещениям уже в первом году, продлит ли клубарь членство?
- Думаю, да.

## Временная оценка посещений

In [ ]:
# Год членства и количество посещенных встреч теми, кто продлят членство (True) и теми, кто не продлит (False) в этот период
# at_meeting_sum  will_come_sum refuse_sum  membership_cut encouraged
# px.violin(members_plus_fmeets_by_mper_id, x='mper_num', y='at_meeting_sum', color='is_continued', box=True, points='suspectedoutliers', title='Количество посещений форум-встреч в период, год членства и продление после него') # , violinmode='overlay'  suspectedoutliers
members_plus_fmeets_by_mper_id.boxplot(x='mper_num', y='refuse_sum', color='is_continued', points='suspectedoutliers', title='Год членства и ... продлившими членство и не продлившими') # , violinmode='overlay'  suspectedoutliers


In [6]:
members_plus_fmeets_by_mper_id.columns

Index(['mper_id', 'stage', 'start_membership_date', 'end_membership_date',
       'user_id', 'contact_id', 'churn', 'membership_days',
       'membership_months', 'membership_cut', 'seq_number', 'meeting_id',
       'at_meeting_sum', 'at_meeting_ewm', 'at_meeting_stab', 'encouraged',
       'mper_num', 'is_continued'],
      dtype='object')

### Склонность посещать встречи (с большим весом последних)

In [11]:
# Год членства и количество посещенных встреч теми, кто продлят членство (True) и теми, кто не продлит (False) в этот период
# px.violin(members_plus_fmeets_by_mper_id, x='mper_num', y='at_meeting_sum', color='is_continued', box=True, points='suspectedoutliers', title='Количество посещений форум-встреч в период, год членства и продление после него') # , violinmode='overlay'  suspectedoutliers
members_plus_fmeets_by_mper_id.boxplot(x='mper_num', y='at_meeting_ewm', color='is_continued', points='suspectedoutliers', color_discrete_sequence=[px.colors.qualitative.Plotly[2], px.colors.qualitative.Plotly[1]], title='Год членства и количество посещенных встреч продлившими членство и не продлившими') # , violinmode='overlay'  suspectedoutliers


Посмотрим, как распределена посещаемость по годам членства у тех, кто продлят (True) и тех, кто не продлят (False) в конкретный период.  
Заметим, что красные коробочки пускают пузыриков с каждым годом всё меньше...

Заглянем внутрь периодов (лет) каждой группы и посмотрим на серединные значения и вероятности количеств посещений

In [13]:
# Год членства и количество посещенных встреч теми, кто продлят членство в этот период
px.violin(members_plus_fmeets_by_mper_id.query('is_continued == True'), x='mper_num', y='at_meeting_ewm', box=True, points='all', color_discrete_sequence=[px.colors.qualitative.Plotly[2]], title='Год членства и количество посещенных встреч теми, кто ПРОДЛИТ членство в этот период') # , violinmode='overlay'  suspectedoutliers


В первый год вероятности посещения все еще тяготеют к 0. 
Но вероятность возле 9 посещений тоже крепка.

Далее вероятности всё более перетекают к 8-9.

In [9]:
# Год членства и количество посещенных встреч теми, кто не продлит членство в этот период
px.violin(members_plus_fmeets_by_mper_id.query('is_continued == False'), x='mper_num', y='at_meeting_ewm', box=True, points='all', color_discrete_sequence=[px.colors.qualitative.Plotly[1]], title='Год членства и количество посещенных встреч теми, кто НЕ ПРОДЛИТ членство в этот период') # , violinmode='overlay'  suspectedoutliers


Первый год у не продливших его все-же довольно насыщен посещениями.  
У не продливших второй год уже крепнет тенденция к не-посещению.  
Далее и сами не-продлевающие постепенно иссякают...

### Нестабильность посещений встреч (с большим весом последних)

In [ ]:
# Год членства и количество посещенных встреч теми, кто продлят членство (True) и теми, кто не продлит (False) в этот период
# at_meeting_sum  will_come_sum refuse_sum  membership_cut encouraged
# px.violin(members_plus_fmeets_by_mper_id, x='mper_num', y='at_meeting_sum', color='is_continued', box=True, points='suspectedoutliers', title='Количество посещений форум-встреч в период, год членства и продление после него') # , violinmode='overlay'  suspectedoutliers
members_plus_fmeets_by_mper_id.boxplot(x='mper_num', y='refuse_sum', color='is_continued', points='suspectedoutliers', title='Год членства и ... продлившими членство и не продлившими') # , violinmode='overlay'  suspectedoutliers


Посмотрим, как распределена посещаемость по годам членства у тех, кто продлят (True) и тех, кто не продлят (False) в конкретный период.  
Заметим, что красные коробочки пускают пузыриков с каждым годом всё меньше...

Заглянем внутрь периодов (лет) каждой группы и посмотрим на серединные значения и вероятности количеств посещений

In [18]:
# Год членства и количество посещенных встреч теми, кто продлят членство в этот период
px.violin(members_plus_fmeets_by_mper_id.query('is_continued == True'), x='mper_num', y='at_meeting_stab', box=True, points='all', color_discrete_sequence=[px.colors.qualitative.Plotly[2]], title='Год членства и НЕСТАБИЛЬНОСТЬ посещений встреч теми, кто ПРОДЛИТ членство в этот период') # , violinmode='overlay'  suspectedoutliers


В первый год вероятности посещения все еще тяготеют к 0. 
Но вероятность возле 9 посещений тоже крепка.

Далее вероятности всё более перетекают к 8-9.

In [19]:
# Год членства и количество посещенных встреч теми, кто не продлит членство в этот период
px.violin(members_plus_fmeets_by_mper_id.query('is_continued == False'), x='mper_num', y='at_meeting_stab', box=True, points='all', color_discrete_sequence=[px.colors.qualitative.Plotly[1]], title='Год членства и НЕСТАБИЛЬНОСТЬ посещений встреч теми, кто НЕ ПРОДЛИТ членство в этот период') # , violinmode='overlay'  suspectedoutliers


Первый год у не продливших его все-же довольно насыщен посещениями.  
У не продливших второй год уже крепнет тенденция к не-посещению.  
Далее и сами не-продлевающие постепенно иссякают...

# Пользователи, посещения формум-встреч, намерения посетить и количество продлений
Используется датасет с группировкой по пользователям

### Количество месяцев в клубе и количество посещенных встреч

In [7]:
# Количество МЕСЯЦЕВ в клубе, количество ПОСЕЩЕННЫХ встреч и ПРОДЛЕНИЯ
px.scatter(members_plus_fmeets_by_mper_contact, x='membership_months', y='at_meeting_sum', color='is_continued', color_continuous_scale='turbo', title='Количество МЕСЯЦЕВ в клубе, количество ПОСЕЩЕННЫХ встреч и ПРОДЛЕНИЯ') # , violinmode='overlay'  suspectedoutliers



На этом графике можно увидеть общую картину посещаемости форум-встреч на протяжении разных периодов членства в клубе.

Каждая точка -- это участник клуба.

Присутствуют 3 признака участников:
- стаж в месяцах (ось 'x')
- количество посещений у участников с таким стажем (ось 'y')
- количество продлений членства (цвет)

Наблюдения:
* У тех, кто был в клюуе менее 10 месяцев почтинет посещений. Они и не продлевали.
* Цветные столбики возле 12, 24, 36, 48, 60 месяцев говорят о том, что их период заканчивался в регулярный срок.
* Остальные рассеянные точки -- видимо, поощренные участники или досрочно прекратившие период.
* Самый разнообразный код по количеству посещений -- 4-й (желтое продление): посетившие 0-10 раз в нем еще заметны.
* В красном продлении таковых уже почти нет.

Можно ли сделать предварительный вывод о наличии связи посещаемости форум-встреч с продлением?
Думаю, да: с увеличением стажа посещаемость уверенно растет и даже отрывается от земли!

### Количество продлений и посещенных встреч

Исследуя датасет с группировкой по пользователям мы видим примерно ту же картину, что и с группировкой по периодам.

In [ ]:
# Количество продлений и количество посещенных встреч в среднем В ПЕРИОД
px.violin(members_plus_fmeets_by_mper_contact, x='is_continued', y=(members_plus_fmeets_by_mper_contact.at_meeting_sum / members_plus_fmeets_by_mper_contact.mper_id), box=True, points='all', color_discrete_sequence=[px.colors.qualitative.Plotly[2], px.colors.qualitative.Plotly[1]], title='Количество ПРОДЛЕНИЙ, среднее количество ПОСЕЩЕННЫХ встреч В ПЕРИОД') # , violinmode='overlay'  suspectedoutliers


В целом, посещаемость с каждым периодом растет и укрепляется.

In [ ]:
# Количество продлений и количество посещенных встреч в среднем В ПЕРИОД
px.violin(members_plus_fmeets_by_mper_contact, x='is_continued', y=(members_plus_fmeets_by_mper_contact.at_meeting_sum / members_plus_fmeets_by_mper_contact.mper_id), color='churn', box=True, points='all', color_discrete_sequence=[px.colors.qualitative.Plotly[2], px.colors.qualitative.Plotly[1]], title='Количество ПРОДЛЕНИЙ, среднее количество ПОСЕЩЕННЫХ встреч В ПЕРИОД и факт ОТТОКа') # , violinmode='overlay'  suspectedoutliers


In [ ]:
# Количество продлений и количество намерений посетить встречи встреч в среднем В ПЕРИОД
# 'at_meeting_sum', 'will_come_sum', 'refuse_sum'
mper_count = members_plus_fmeets_by_mper_contact.mper_id
px.violin(members_plus_fmeets_by_mper_contact, x='is_continued',\
    y=[\
    # (members_plus_fmeets_by_mper_contact.at_meeting_sum / mper_count),\
    (members_plus_fmeets_by_mper_contact.will_come_sum / mper_count),\
    # (members_plus_fmeets_by_mper_contact.refuse_sum / mper_count),\
    ],\
    box=True, points='all', color_discrete_sequence=[px.colors.qualitative.Plotly[3]], title='Количество ПРОДЛЕНИЙ, среднее количество НАМЕРЕНИЙ посетить встречи В ПЕРИОД') # , violinmode='overlay'  suspectedoutliers


In [ ]:
# Количество продлений и количество отметок "НЕ ПРИШЕЛ" в среднем В ПЕРИОД
# 'at_meeting_sum', 'will_come_sum', 'refuse_sum'
mper_count = members_plus_fmeets_by_mper_contact.mper_id
px.violin(members_plus_fmeets_by_mper_contact, x='is_continued',\
    y=[\
    # (members_plus_fmeets_by_mper_contact.at_meeting_sum / mper_count),\
    # (members_plus_fmeets_by_mper_contact.will_come_sum / mper_count),\
    (members_plus_fmeets_by_mper_contact.refuse_sum / mper_count),\
    ],\
    box=True, points='all', color_discrete_sequence=[px.colors.qualitative.Plotly[1]], title='Количество ПРОДЛЕНИЙ и количество отметок "НЕ ПРИШЕЛ" в среднем В ПЕРИОД') # , violinmode='overlay'  suspectedoutliers


А вот отметки "не пришел" в среднем не высоки, но стабильны!

### Корреляции признаков

Посмотрим на корреляции признаков, касающихся посещений и продлений при такой группировке (по участнику).

Самая слабая корреляция с другими признаками наблюдается у переменной "приду".  
Мы видели, что не многие осстаются в этом статусе.

Самая уверенная корреляция -- у посещенных встреч с количеством продлений, как мы и предполагали (при слабой , но также выделяющейся корреляции продления отдельных периодов при группировке по ним).

Это наблюдение не оставляет последних сомнений в связи посещения форум-втсреч и продления членства в клубе.

In [76]:
# Корреляция между количественными призаками
corr_df = members_plus_fmeets_by_mper_contact[['at_meeting_sum', 'will_come_sum', 'refuse_sum', 'seq_number', 'is_continued']].corr().round(2) #   is_continued   churn
corr_df

px.imshow(corr_df, text_auto=True, aspect="auto", color_continuous_scale='turbo', width=500, height=400, color_continuous_midpoint=0) 

In [79]:
# Это признак, сигнализирующий нолями о не отмеченных в таблице встреч учатсниках!
members_plus_fmeets_by_mper_contact.seq_number.value_counts()

seq_number
0.0     370
10.0    106
7.0      92
9.0      86
6.0      85
8.0      82
4.0      66
1.0      65
5.0      63
2.0      55
3.0      53
Name: count, dtype: int64

# Важность признаков

In [47]:
# members_plus_fmeets_by_mper_id    members_plus_fmeets_by_mper_contact 
members_plus_fmeets_by_mper_id.columns
# members_plus_fmeets_by_mper_id.columns

Index(['mper_id', 'stage', 'start_membership_date', 'end_membership_date',
       'user_id', 'contact_id', 'churn', 'membership_days',
       'membership_months', 'membership_cut', 'seq_number', 'meeting_id',
       'at_meeting_sum', 'will_come_sum', 'refuse_sum', 'encouraged',
       'mper_num', 'is_continued'],
      dtype='object')

Полный вариант
```
Explained Variance Ratio: [0.98875201 0.00955495]
Feature Weights:
Component 1 :
   membership_days : 0.998958410358575
   membership_months : 0.03399776155351563
   meeting_id : 0.01856013103883941
   at_meeting_sum : 0.016481176967803487
   seq_number : 0.01636598486882052
   encouraged : 0.003622195037591119
   refuse_sum : 0.0033736794907263832
   membership_cut : 0.0029354081788776566
   is_continued : 0.0024895884096887482
   mper_num : 0.0016913418398308505
   will_come_sum : 0.0003408005690696619
Component 2 :
   meeting_id : 0.5944131394798342
   seq_number : 0.5859501375744856
   at_meeting_sum : 0.5403166849270985
   refuse_sum : 0.08965217120560397
   mper_num : 0.0425719090175946
   membership_days : 0.029766491096143658
   will_come_sum : 0.018067738217179538
   is_continued : 0.01804021449552056
   membership_months : 0.0022344870348687262
   encouraged : 0.0007221264178635979
   membership_cut : 7.580170963018482e-05
```

In [48]:
from sklearn.decomposition import PCA


In [20]:
data = members_plus_fmeets_by_mper_id[[
    # 'start_membership_date',
    'membership_days',
    'membership_months', 'membership_cut', 'seq_number', 'meeting_id',
    'at_meeting_sum', 'at_meeting_ewm', 'at_meeting_stab', 'encouraged',
    'mper_num', 'is_continued'
]]


In [21]:
from sklearn.decomposition import PCA

# Создание экземпляра класса PCA и указание количества компонент
pca = PCA(n_components=2)

# Применение PCA к данным
data_pca = pca.fit_transform(data)

# Вывод объясненной дисперсии каждой компоненты
explained_variance_ratio = pca.explained_variance_ratio_
print('Explained Variance Ratio:', explained_variance_ratio)

# Получение весов признаков в первых двух компонентах
feature_weights = np.abs(pca.components_)
print('Feature Weights:')
for i, weights in enumerate(feature_weights):
    # Сортировка индексов по убыванию весов
    features_sorted = sorted(range(len(weights)), key=lambda x: weights[x], reverse=True)
    
    # Получение названий признаков
    feature_names = data.columns.values
    
    # Вывод названий признаков с весами в первых двух компонентах
    print('Component', i+1, ':')
    for feature_index in features_sorted:
        print('  ', feature_names[feature_index], ':', weights[feature_index])

Explained Variance Ratio: [0.99079762 0.00723623]
Feature Weights:
Component 1 :
   membership_days : 0.9990748998551698
   membership_months : 0.03400179972326028
   meeting_id : 0.01851790273494112
   at_meeting_sum : 0.016445708080517794
   seq_number : 0.0068022824813831805
   encouraged : 0.003622765360698516
   membership_cut : 0.0029357275613047496
   is_continued : 0.002488516334701866
   mper_num : 0.0016898339039551696
   at_meeting_ewm : 0.0015792830425103788
   at_meeting_stab : 0.000540092365270633
Component 2 :
   meeting_id : 0.6784646449158062
   at_meeting_sum : 0.6311714334891014
   seq_number : 0.36735206073594456
   at_meeting_ewm : 0.052426752675604586
   mper_num : 0.04839116494400447
   membership_days : 0.025452608293030993
   is_continued : 0.020495105268314814
   at_meeting_stab : 0.013900083161715683
   membership_months : 0.0023642927694103033
   encouraged : 0.0019624171867987095
   membership_cut : 5.913201584099512e-05


In [89]:
# members_plus_fmeets_by_mper_contact не отличается

from sklearn.decomposition import PCA

# Создание экземпляра класса PCA и указание количества компонент
pca = PCA(n_components=2)

# Применение PCA к данным
data_pca = pca.fit_transform(data)

# Вывод объясненной дисперсии каждой компоненты
explained_variance_ratio = pca.explained_variance_ratio_
print('Explained Variance Ratio:', explained_variance_ratio)

# Получение весов признаков в первых двух компонентах
feature_weights = np.abs(pca.components_)
print('Feature Weights:')
for i, weights in enumerate(feature_weights):
    # Сортировка индексов по убыванию весов
    features_sorted = sorted(range(len(weights)), key=lambda x: weights[x], reverse=True)
    
    # Получение названий признаков
    feature_names = data.columns.values
    
    # Вывод названий признаков с весами в первых двух компонентах
    print('Component', i+1, ':')
    for feature_index in features_sorted:
        print('  ', feature_names[feature_index], ':', weights[feature_index])

Explained Variance Ratio: [9.99406650e-01 4.85672036e-04]
Feature Weights:
Component 1 :
   membership_days : 0.9987937137851672
   membership_months : 0.03275586911768927
   meeting_id : 0.02758884936521199
   at_meeting_sum : 0.02297564550615896
   seq_number : 0.004623173421163569
   refuse_sum : 0.0042297275715627875
   is_continued : 0.0029996787645628128
   encouraged : 0.0007674667808392537
   will_come_sum : 0.0005415320933424755
   membership_cut : 2.7983690976286604e-05
Component 2 :
   meeting_id : 0.706383834251114
   at_meeting_sum : 0.678310639398179
   seq_number : 0.18339979720640015
   refuse_sum : 0.07354950385779718
   membership_days : 0.03628301833920936
   will_come_sum : 0.020908895146499285
   is_continued : 0.010307764897514306
   encouraged : 0.003211050951489677
   membership_months : 0.0009902488519015762
   membership_cut : 0.000599400214038065


# Что дальше...

Мне кажется, полезно в исследованиях сосредоточиться на интересах участников, чтобы более эффекитвно предлагать 
* то, что им подходит и 
* то, что для них более ценно.

Здесь может помочь изучение профилей участников/пользователей и их активностей в приложении.
Посмотреть, как группируются участники по профилям и активностям.
Посмотреть динамику активностей и благодарностей.
Посмотреть, как всё это соотносится с форум-встречами и с оттоком.


В дальнейшем можно думать о персональной системе рекомендаций материалов и создании персональных сводок по новостям и аналитике в приложении.

# Кластеризация периодного датасета

### 1 всё, 3 избранных признака для каждой компонениы

In [ ]:
data = members_plus_fmeets_by_mper_id[[
    # 'start_membership_date',
    'membership_days',
    'membership_months', 'membership_cut', 'seq_number', 'meeting_id',
    'at_meeting_sum', 'will_come_sum', 'refuse_sum', 'encouraged',
    'mper_num', 'is_continued'
]]

In [74]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
import plotly.express as px

# Создание экземпляра класса PCA и указание количества компонент
pca = PCA(n_components=2)

# Применение PCA к данным
data_pca = pca.fit_transform(data)

# Получение весов признаков в первых двух компонентах
feature_weights = np.abs(pca.components_)

# Отбор по 3 наиболее эффективных признака для каждой оси
selected_features = []
for weights in feature_weights:
    # Сортировка индексов признаков по убыванию весов
    features_sorted = sorted(range(len(weights)), key=lambda x: weights[x], reverse=True)
    
    # Получение названий признаков
    feature_names = data.columns.values
    
    # Выбор первых 3 признаков с наибольшими весами
    selected_features.append([feature_names[feature_index] for feature_index in features_sorted[:3]])
    
# Применение кластеризации DBSCAN
dbscan = DBSCAN()
labels = dbscan.fit_predict(data_pca)

# Создание DataFrame с данными кластера
cluster_data = pd.DataFrame(data_pca, columns=['Component 1', 'Component 2'])
cluster_data['Cluster'] = labels

# Создание графика с цветовым разделением кластеров
fig = px.scatter(cluster_data, x='Component 1', y='Component 2', color='Cluster')
fig.show()

# Вывод признаков в каждом кластере
for cluster_id in np.unique(labels):
    cluster_features = selected_features[cluster_id]
    print(f"Кластер {cluster_id}: {cluster_features}")


Кластер -1: ['meeting_id', 'at_meeting_sum', 'seq_number']
Кластер 0: ['membership_days', 'membership_months', 'meeting_id']
Кластер 1: ['meeting_id', 'at_meeting_sum', 'seq_number']


IndexError: list index out of range

In [65]:
len(selected_features), np.unique(labels)

(2, array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int64))

In [63]:
cluster_data

,Component 1,Component 2,Cluster
0,-2.644134,-3.695609,-1
1,12.392915,-3.795657,0
2,12.340485,-1.992999,0
3,12.315293,-1.173648,0
4,159.598827,-4.868806,-1
...,...,...,...
2282,-323.048308,-0.059490,-1
2283,-285.057273,0.988727,-1
2284,-327.078139,-0.180791,-1
2285,-350.054183,-0.865420,-1


### 2

In [80]:
data.columns

Index(['membership_days', 'membership_months', 'membership_cut', 'seq_number',
       'meeting_id', 'at_meeting_sum', 'will_come_sum', 'refuse_sum',
       'encouraged', 'mper_num', 'is_continued'],
      dtype='object')

In [85]:
from sklearn.cluster import DBSCAN
import plotly.express as px

# Выбор трех признаков для кластеризации
data_selected = data[['membership_months', 'at_meeting_sum', 'mper_num']]

# Применение кластеризации DBSCAN
dbscan = DBSCAN()
labels = dbscan.fit_predict(data_selected)

# Создание DataFrame с данными кластера
cluster_data = data_selected.copy()
cluster_data['Cluster'] = labels

# Создание графика с цветовым разделением кластеров
fig = px.scatter_3d(cluster_data, x='membership_months', y='at_meeting_sum', z='mper_num', color='Cluster', height=800)
fig.show()


In [ ]:
data = members_plus_fmeets_by_mper_id[[
    # 'start_membership_date',
    'membership_days',
    'membership_months', 'membership_cut', 'seq_number', 'meeting_id',
    'at_meeting_sum', 'will_come_sum', 'refuse_sum', 'encouraged',
    'mper_num', 'is_continued'
]]

In [ ]:
from sklearn.cluster import DBSCAN
import plotly.express as px

# Выбор трех признаков для кластеризации
data_selected = data[['mper_num', 'at_meeting_sum']]

# Применение кластеризации DBSCAN
dbscan = DBSCAN()
labels = dbscan.fit_predict(data_selected)

# Создание DataFrame с данными кластера
cluster_data = data_selected.copy()
cluster_data['Cluster'] = labels

# Создание графика с цветовым разделением кластеров
fig = px.scatter(cluster_data, x='mper_num', y='at_meeting_sum', color='Cluster', height=800)
fig.show()


In [86]:
members_plus_fmeets_by_mper_contact.columns

Index(['start_membership_date', 'end_membership_date', 'user_id', 'contact_id',
       'encouraged', 'mper_id', 'churn', 'is_continued', 'membership_days',
       'membership_months', 'membership_cut', 'seq_number', 'meeting_id',
       'at_meeting_sum', 'will_come_sum', 'refuse_sum'],
      dtype='object')

In [88]:
data = members_plus_fmeets_by_mper_contact[[
    # 'start_membership_date',
    'membership_days',
    'membership_months', 'membership_cut', 'seq_number', 'meeting_id',
    'at_meeting_sum', 'will_come_sum', 'refuse_sum', 'encouraged',
    'is_continued'
]]

In [91]:
from sklearn.cluster import DBSCAN
import plotly.express as px

# Выбор трех признаков для кластеризации
data_selected = data[['membership_months', 'at_meeting_sum', 'is_continued']]

# Применение кластеризации DBSCAN
dbscan = DBSCAN()
labels = dbscan.fit_predict(data_selected)

# Создание DataFrame с данными кластера
cluster_data = data_selected.copy()
cluster_data['Cluster'] = labels

# Создание графика с цветовым разделением кластеров
fig = px.scatter_3d(cluster_data, x='membership_months', y='at_meeting_sum', z='is_continued', color='Cluster', height=800)
fig.show()


Кластеризация (как вид машинного обучения) по двум важным признакам и признаку количества продлений не дала вразумительных результатов.

Но на графике мы можем увидеть